In [ ]:
%reset
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import numpy as np
import matplotlib.pyplot as plt
import torch

import defmod as dm

torch.set_default_tensor_type(torch.DoubleTensor)

In [ ]:
source = dm.sampling.load_and_sample_greyscale("../data/density_a.png", threshold=0.5, centered=True)
target = dm.sampling.load_and_sample_greyscale("../data/density_b.png", threshold=0.5, centered=True)

In [ ]:
aabb = dm.usefulfunctions.AABB.build_from_points(source[0])

In [ ]:
sigma = 0.1
x, y = torch.meshgrid([torch.arange(aabb.xmin-sigma, aabb.xmax+sigma, step=sigma), torch.arange(aabb.ymin-sigma, aabb.ymax+sigma, step=sigma)])
gd = dm.usefulfunctions.grid2vec(x, y).contiguous().view(-1)

In [ ]:
dm.usefulfunctions.plotTensorScatter(source, alpha=0.4)
dm.usefulfunctions.plotTensorScatter(target, alpha=0.4)
plt.plot(gd.view(-1, 2)[:, 1].numpy(), gd.view(-1, 2)[:, 0].numpy(), '.')
plt.show()

In [ ]:
my_model = dm.models.ModelTranslationModuleRegistration(2, source, sigma, gd, True)
costs = my_model.fit(target, max_iter=400, l=80., lr=1e-4, log_interval=5)

In [ ]:
out = my_model()
out_gd, _ = my_model.shoot_list()
%matplotlib qt5
plt.subplot(1, 2, 1)
dm.usefulfunctions.plotTensorScatter(target, alpha=0.4)
dm.usefulfunctions.plotTensorScatter(out, alpha=0.4)
plt.plot(out_gd[1].view(-1, 2).detach()[:, 1].numpy(), out_gd[1].view(-1, 2).detach()[:, 0].numpy(), '.')
plt.subplot(1, 2, 2)
dm.usefulfunctions.plotTensorScatter(target, alpha=0.4)
dm.usefulfunctions.plotTensorScatter(source, alpha=0.4)
plt.show()


In [ ]:
plt.plot(range(0, len(costs)), costs)
plt.show()